In [1]:
import cv2
from cv2 import aruco
import numpy as np
import pandas as pd
import os
import glob

In [2]:
# ArUco dictionary generate
aruco_dict = aruco.Dictionary_get(aruco.DICT_5X5_100)
# random marker board generate
board = aruco.CharucoBoard_create(7, 6, 1, .8, aruco_dict)
# board image generate
imboard = board.draw((2000, 2000))

# calibrate camera parameter
calibration_flags = (cv2.CALIB_USE_INTRINSIC_GUESS +
             cv2.CALIB_RATIONAL_MODEL + cv2.CALIB_FIX_ASPECT_RATIO)
calibration_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-9)

In [3]:
# Extract individual images stored in a given directory
images = glob.glob('./calib_new_testimage1/*.jpg')
captured_images = []

In [4]:
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

# Find checkerboard 2D coordinates

In [5]:
allCorners = []
allIds = []

decimator = 0
criteria = (cv2.TERM_CRITERIA_EPS +
                cv2.TERM_CRITERIA_MAX_ITER, 100, 0.00001)

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(
            gray, aruco_dict)

    if len(corners) > 0:
        # SUB PIXEL DETECTION
        for corner in corners:
            # Checkerboard corner improvements
            cv2.cornerSubPix(gray, corner,
                                winSize=(3, 3),
                                zeroZone=(-1, -1),
                                criteria=criteria)
        res2 = cv2.aruco.interpolateCornersCharuco(
            corners, ids, gray, board)
        if res2[1] is not None and res2[2] is not None and len(res2[1]) > 3 and decimator % 1 == 0:
            allCorners.append(res2[1])
            allIds.append(res2[2])

    decimator += 1
imsize = gray.shape


# Camera calibration

In [6]:
cameraMatrixInit = np.array([[1000.,    0., imsize[0]/2.],
                                 [0., 1000., imsize[1]/2.],
                                 [0.,    0.,           1.]])

distCoeffsInit = np.zeros((5, 1))
flags = (cv2.CALIB_USE_INTRINSIC_GUESS +
         cv2.CALIB_RATIONAL_MODEL + cv2.CALIB_FIX_ASPECT_RATIO)
# flags = (cv2.CALIB_RATIONAL_MODEL)
(ret, camera_matrix, distortion_coefficients0,
rotation_vectors, translation_vectors,
stdDeviationsIntrinsics, stdDeviationsExtrinsics,
perViewErrors) = cv2.aruco.calibrateCameraCharucoExtended(
    charucoCorners=allCorners,
    charucoIds=allIds,
    board=board,
    imageSize=imsize,
    cameraMatrix=cameraMatrixInit,
    distCoeffs=distCoeffsInit,
    flags=calibration_flags,
    criteria=calibration_criteria)

## Intrinsic camera matrix

In [7]:
print("Camera matrix : \n") 
print(camera_matrix)

Camera matrix : 

[[4.67146077e+03 0.00000000e+00 1.38924359e+03]
 [0.00000000e+00 4.67146077e+03 1.74117421e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


## Lens distortion coefficients

In [8]:
print("Distortion coefficients : \n") 
print(distortion_coefficients0)

Distortion coefficients : 

[[-2.40841702e+01]
 [ 1.87806516e+02]
 [-4.09401417e-02]
 [ 2.55104022e-02]
 [-1.34885758e+02]
 [-2.38359469e+01]
 [ 1.82013936e+02]
 [-9.24638217e+01]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]


## Rotation matrix

In [9]:
print("rvecs : \n") 
print(rotation_vectors)

rvecs : 

(array([[ 3.03254202],
       [-0.11009948],
       [ 0.05843064]]), array([[3.05458146],
       [0.0240563 ],
       [0.13344233]]), array([[ 3.04836692],
       [-0.05787299],
       [ 0.12677816]]), array([[3.04665887],
       [0.08260337],
       [0.12515124]]), array([[3.04134252],
       [0.02007869],
       [0.09473734]]), array([[ 3.04758497],
       [-0.0611865 ],
       [ 0.11574196]]), array([[3.04269599],
       [0.10177816],
       [0.10645557]]), array([[ 3.04827236e+00],
       [-1.91938148e-03],
       [ 9.00160931e-02]]), array([[ 3.05222751],
       [-0.05146169],
       [ 0.09792406]]), array([[ 3.05015022],
       [-0.08630619],
       [ 0.0911781 ]]), array([[3.04573694],
       [0.015571  ],
       [0.10578046]]), array([[3.03102355],
       [0.08269744],
       [0.10055215]]))


## Translation matrix

In [10]:
print("tvecs : \n")
print(translation_vectors)

tvecs : 

(array([[-1.64475073],
       [ 1.05634872],
       [15.50734538]]), array([[-3.54314591],
       [ 0.67642852],
       [15.29067668]]), array([[-3.40225044],
       [ 0.50694227],
       [15.34144501]]), array([[-3.24032822],
       [ 0.23523688],
       [15.34432266]]), array([[-2.56581183],
       [ 0.66598325],
       [15.51299736]]), array([[-2.60174008],
       [ 0.68658213],
       [15.39463969]]), array([[-2.88272422],
       [ 0.45178947],
       [15.38152992]]), array([[-2.47084436],
       [ 0.05264239],
       [15.42861496]]), array([[-2.39021367],
       [ 0.31501947],
       [15.4319077 ]]), array([[-2.39939412],
       [ 0.5071712 ],
       [15.45150196]]), array([[-2.67234295],
       [ 0.83350209],
       [15.41634037]]), array([[-2.60798266],
       [ 0.81287317],
       [15.38804194]]))


# Undistort

In [11]:
# undistort
dir="calib_new_testimage1"
createDirectory("undistort_new1")

for i in os.listdir(dir):
    img=cv2.imread(dir+'/'+i)
    img_undist = cv2.undistort(img, camera_matrix, distortion_coefficients0, None)
    concatenated_image = cv2.hconcat([img, img_undist])
    concatenated_image = cv2.resize(concatenated_image, (0,0), fx=0.5, fy=0.5)
    cv2.imshow("undistorted", concatenated_image)
    cv2.imwrite("undistort_new1/undist_{0}.png".format(i), concatenated_image)
    key = cv2.waitKey(0)
    if key == 27:
        cv2.destroyAllWindows()